In [1]:
!pwd

/home/user/dmlee/summarization_tf1.14/models/official/transformer


In [2]:
%cd /home/user/dmlee/summarization_tf1.14/models



/home/user/dmlee/summarization_tf1.14/models


In [3]:
from absl import app as absl_app


In [4]:

DATA_DIR="/home/user/dmlee/summarization_tf1.14/t2t_data"
MODEL_DIR="/home/user/dmlee/summarization_tf1.14/models/CLS_4_summ_512_128_lr0.15_wrm8k_rmlen2048_base"
DECODE_VOCAB_FILE="/home/user/dmlee/summarization_tf1.14/t2t_data/vocab.txt"

model_dir = MODEL_DIR
param_set = "base"
vocab_file = DECODE_VOCAB_FILE
input_file = DATA_DIR+"/sample_src_cls.txt"
input_target_file = DATA_DIR+"/sample_tgt_cls.txt"


In [5]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os

# pylint: disable=g-bad-import-order
from absl import app as absl_app
from absl import flags
import tensorflow as tf
# pylint: enable=g-bad-import-order
from official.transformer.utils import tokenizer_cls as tokenizer

from official.utils.flags import core as flags_core
tf.enable_eager_execution()

_DECODE_BATCH_SIZE = 1
_EXTRA_DECODE_LENGTH = 0
_BEAM_SIZE = 5
_ALPHA = 0.8


/home/user/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/user/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/user/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/user/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWa

In [6]:
def _encode_src_and_add_eos(line, subtokenizer):
    """Encode line with subtokenizer, and add EOS id to the end."""
    ids = subtokenizer.encode(line)
    if len(ids)>511:
        ids = subtokenizer.encode(line)[:511]

    tf.logging.info(ids)
    return ids + [tokenizer.EOS_ID]

def _encode_tgt_and_add_eos(line, subtokenizer):
    """Encode line with subtokenizer, and add EOS id to the end."""
    ids = subtokenizer.encode(line)
    if len(ids)>127:
        ids = subtokenizer.encode(line)[:127]

    tf.logging.info(ids)
    return ids + [tokenizer.EOS_ID]


In [7]:
def plot_attention_weights(attention, sentence, result, layer, subtokenizer):
    fig = plt.figure(figsize=(16, 8))

    sentence = _encode_src_and_add_eos(sentence, subtokenizer)
    result = _encode_src_and_add_eos(result, subtokenizer)
    
    attention = tf.squeeze(attention[layer], axis=0)

    for head in range(attention.shape[0]):
        ax = fig.add_subplot(2, 4, head+1)

        # plot the attention weights
        ax.matshow(attention[head][:-1, :], cmap='viridis')

        fontdict = {'fontsize': 10}

        ax.set_xticks(range(len(sentence)+2))
        ax.set_yticks(range(len(result)))

        ax.set_ylim(len(result)-1.5, -0.5)

        ax.set_xticklabels(
            subtokenizer.decode_withCLS(sentence) , 
            fontdict=fontdict, rotation=90)

        ax.set_yticklabels(subtokenizer.decode_withCLS(result), 
                           fontdict=fontdict)

        ax.set_xlabel('Head {}'.format(head+1))

    plt.tight_layout()
    plt.show()

In [8]:
def _get_sorted_inputs(filename, target_filename):
    """Read and sort lines from the file sorted by decreasing length.
    
    Args:
    filename: String name of file to read inputs from.
    Returns:
    Sorted list of inputs, and dictionary mapping original index->sorted index
    of each element.
    """
    with tf.io.gfile.GFile(filename) as f:
        records = f.read().split("\n")
        inputs = [record.strip() for record in records]
        if not inputs[-1]:
            inputs.pop()

    with tf.io.gfile.GFile(target_filename) as f:
        tgt_records = f.read().split("\n")
        tgt_inputs = [record.strip() for record in tgt_records]
        if not tgt_inputs[-1]:
            tgt_inputs.pop()

    input_lens = [(i, len(line.split())) for i, line in enumerate(inputs)]
    sorted_input_lens = sorted(input_lens, key=lambda x: x[1], reverse=True)

    sorted_inputs = [None] * len(sorted_input_lens)
    sorted_tagets = [None] * len(sorted_input_lens)

    sorted_index = [0] * len(sorted_input_lens)

    for i, (index, _) in enumerate(sorted_input_lens):
        sorted_inputs[i] = inputs[index]
        sorted_tagets[i] = tgt_inputs[index]
        sorted_index[index] = i

    return sorted_inputs, sorted_tagets, sorted_index

In [12]:

def translate_file(estimator, 
     subtokenizer, input_file, input_target_file):
    """Translate lines in file, and save to output file if specified.

    Args:
    estimator: tf.Estimator used to generate the translations.
    subtokenizer: Subtokenizer object for encoding and decoding source and
    translated lines.
    input_file: file containing lines to translate
    output_file: file that stores the generated translations.
    print_all_translations: If true, all translations are printed to stdout.

    Raises:
    ValueError: if output file is invalid.
    """
    batch_size = _DECODE_BATCH_SIZE

    # Read and sort inputs by length. Keep dictionary (original index-->new index
    # in sorted list) to write translations in the original order.
    sorted_inputs, sorted_input_targets, sorted_keys = _get_sorted_inputs(input_file, input_target_file)
    num_decode_batches = (len(sorted_inputs) - 1) // batch_size + 1

    inputs = zip( sorted_inputs, sorted_input_targets )


    def input_generator():
        """Yield encoded strings from sorted_inputs."""
        for i, (src_line, tgt_line) in enumerate(inputs):
            if i % batch_size == 0:
                batch_num = (i // batch_size) + 1

                tf.logging.info("Decoding batch %d out of %d." %
                                (batch_num, num_decode_batches))
            yield _encode_src_and_add_eos(src_line, subtokenizer), _encode_tgt_and_add_eos(tgt_line, subtokenizer)

    def input_fn():
        """Created batched dataset of encoded inputs."""
        ds = tf.data.Dataset.from_generator(
            input_generator, (tf.int64, tf.int64), (tf.TensorShape([None]), tf.TensorShape([None])) )#define thatsequence length
        ds = ds.padded_batch(batch_size, ([512], [128]) ,drop_remainder=True)#define the sequence length 

        return ds
    
    for x in input_fn():
        print(x)
    for i, prediction in enumerate(estimator.train(input_fn)):
        attentions = prediction["predictions"]
        print(attentions)
        sorted_inputs[i]


In [13]:
def main():
    from official.transformer import transformer_main_cls_vis as transformer_main 

    tf.logging.set_verbosity(tf.logging.INFO)


    subtokenizer = tokenizer.Subtokenizer(vocab_file)
    # Set up estimator and params
    params = transformer_main.PARAMS_MAP[param_set]
    params["vocab_file"]=vocab_file
    params["beam_size"] = _BEAM_SIZE
    params["alpha"] = _ALPHA

    params["extra_decode_length"] = _EXTRA_DECODE_LENGTH
    params["batch_size"] = _DECODE_BATCH_SIZE
    params["static_batch"] = True


    estimator = tf.estimator.Estimator(
    model_fn=transformer_main.model_fn, model_dir=model_dir,
    params=params)

    translate_file(estimator, subtokenizer, input_file, input_target_file)


In [14]:
main()

INFO:tensorflow:Initializing Subtokenizer from file /home/user/dmlee/summarization_tf1.14/t2t_data/vocab.txt.
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/home/user/dmlee/summarization_tf1.14/models/CLS_4_summ_512_128_lr0.15_wrm8k_rmlen2048_base', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f27a6371400>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_eval

INFO:tensorflow:[2, 19, 12549, 13847, 6731, 4389, 1696, 9576, 6869, 956, 32, 43, 14502, 15, 24, 1607, 643, 30, 1045, 31, 2332, 7, 123, 62, 13, 55, 1679, 7236, 15818, 2, 19, 9576, 6869, 956, 10213, 529, 10213, 1956, 3, 1638, 690, 546, 592, 6, 1632, 961, 4311, 10, 7, 836, 1196, 10, 58, 24, 313, 4573, 298, 96, 645, 15818, 2, 19, 20, 228, 322, 2007, 13, 614, 612, 38, 6, 5528, 36, 5900, 1168, 49, 23, 140, 24, 737, 6724, 154, 12549, 54, 11, 1826, 13, 333, 1558, 8, 3037, 9084, 15818, 2, 19, 12549, 13847, 6731, 4389, 1696, 9576, 6869, 956, 32, 43, 14502, 15, 24, 1607, 643, 30, 1045, 2, 19, 61, 53, 33, 979, 15, 3, 487, 8, 3, 333, 9, 32, 117, 1746, 516, 6, 3, 3954, 346, 3, 309, 8, 3, 2769, 333, 15818, 2, 19, 12549, 54, 11, 4817, 1355, 707, 10213, 14817, 20784, 10213, 22, 26014, 30942, 1223, 1696, 32, 5860, 8389, 1805, 188, 3, 2815, 1216, 17, 1169, 24, 333, 96, 645, 15818, 2, 19, 2916, 30942, 1223, 61, 32, 717, 26524, 280, 10, 700, 9, 10, 24, 138, 13, 3, 13, 831, 741, 6, 3473, 79, 15, 24, 643, 9,

AttributeError: in converted code:

    /home/user/dmlee/summarization_tf1.14/models/official/transformer/model/transformer_vis.py:384 call
        enc_dec_attns[f"layer_{n}"]=enc_dec_attention_layer.attention_map

    AttributeError: 'PrePostProcessingWrapper' object has no attribute 'attention_map'
